In [1]:
import yfinance as yf
import pandas as pd
import os # Biblioteca para interagir com o sistema de arquivos
#python -m jupyter nbconvert --to python app.ipynb or jupyter nbconvert --to python app.ipynb or python -m nbconvert --to python teste.ipynb
# pip freeze > requirements.txt
TICKERS = [
    "BOVA11.SA", "SMAL11.SA", "IVVB11.SA",
    "PETR4.SA", "VALE3.SA", "ITUB4.SA",
    "SPY", "QQQ", "IWM", "EEM", "GLD", "TLT",
    "BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "ADA-USD"
]

NOME_ARQUIVO = "precos_historicos_ativos_mistos.csv"
FORCAR_ATUALIZACAO = False  # Mude para True se quiser baixar dados novos hoje

def baixar_dados(tickers, start="2015-01-01", end=None, intervalo="1d"):
    print("Iniciando download do Yahoo Finance...")
    dados = yf.download(
        tickers,
        start=start,
        end=end,
        interval=intervalo,
        auto_adjust=True,
        progress=False 
    )

    # Tratamento para MultiIndex (exatamente como no seu código original)
    if isinstance(dados.columns, pd.MultiIndex):
        if 'Adj Close' in dados.columns.levels[0]:
            dados = dados['Adj Close']
        elif 'Close' in dados.columns.levels[0]:
            dados = dados['Close']
        else:
            raise ValueError(f"Não encontrei 'Adj Close' nem 'Close'.")
    
    dados = dados.reindex(columns=tickers)
    dados = dados.dropna(how='all')
    dados = dados.fillna(pd.NA)
    
    return dados

# --- LÓGICA DE VERIFICAÇÃO DO ARQUIVO ---

# Verifica se o arquivo existe E se não pedimos para forçar atualização
if os.path.exists(NOME_ARQUIVO) and not FORCAR_ATUALIZACAO:
    print("Arquivo encontrado! Carregando dados locais...")
    
    # index_col=0: Diz que a primeira coluna (Datas) é o índice
    # parse_dates=True: Converte as strings de data volta para objetos de data (datetime)
    precos = pd.read_csv(NOME_ARQUIVO, index_col=0, parse_dates=True)
    
else:
    print("Arquivo não encontrado ou atualização forçada. Baixando...")
    precos = baixar_dados(TICKERS)
    # Salva imediatamente para usar na próxima vez
    precos.to_csv(NOME_ARQUIVO)
    print(f"Dados salvos em {NOME_ARQUIVO}")

# Validação visual
print(precos.head())
print(precos.shape)


Arquivo encontrado! Carregando dados locais...
            BOVA11.SA  SMAL11.SA  IVVB11.SA  PETR4.SA   VALE3.SA  ITUB4.SA  \
Date                                                                         
2015-01-01        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-02  47.259998  52.020000  55.799999  2.572509  10.505502  9.372999   
2015-01-03        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-04        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-05  46.320000  50.549999  55.750000  2.352637  10.347521  9.420101   

                   SPY        QQQ         IWM        EEM         GLD  \
Date                                                                   
2015-01-01         NaN        NaN         NaN        NaN         NaN   
2015-01-02  171.093704  94.906532  103.315140  30.789457  114.080002   
2015-01-03         NaN        NaN         NaN        NaN         NaN   
2015-01-04         NaN        NaN         NaN 

In [2]:
NUM_ATIVOS = len(TICKERS)
print("NUM_ATIVOS =", NUM_ATIVOS)

NUM_ATIVOS = 17


(1 + series).cumprod()

Funções envolvidas:
• cumprod() é do pandas.
• Ela calcula o produto acumulado dos valores ao longo da série.

O que você está fazendo aqui:

1 + series transforma seus retornos diários em fatores de crescimento.

Exemplo bobo:

retorno 5% → 1 + 0.05 = 1.05

retorno -3% → 1 + (-0.03) = 0.97

Depois o .cumprod() pega esses fatores e multiplica tudo em sequência, construindo um preço acumulado como se você estivesse investindo dia após dia sem tirar nada.

Pense nisso como reconstruir o gráfico de um portfólio a partir dos retornos.

2. cumulative.cummax()

• cummax() também é do pandas.
• Ela calcula o máximo acumulado ao longo do tempo.

É tipo isso:

Dia 1: valor 100 → máximo = 100
Dia 2: valor 105 → máximo = 105
Dia 3: valor 102 → máximo continua = 105
Dia 4: valor 110 → máximo vira = 110
Dia 5: valor 108 → máximo fica = 110

Isso representa o maior valor histórico atingido até cada ponto da série.

1. if d < -0.20:

Se o drawdown é pior que –20%, ele classifica como bear.
Por quê 20%?
Porque o mercado tradicional considera drawdown de 20% ou mais como queda forte, típica de bear markets.

2. elif r > 0.003 and d > -0.05:

Se retorno diário é maior que 0.3% e o drawdown é maior que –5%, ele chama de bull.

Isso tenta capturar momentos em que:

• o mercado está subindo com força moderada
• está perto do topo histórico
• não está em queda recente

É uma definição operacional, simples, nada sagrado.

3. elif v > 0.25:

Volatilidade acima de 25% anualizada é considerada alta.
Mercado instável, muito sobe e desce, clima de incerteza.

Aquela fase em que tudo parece saltar igual milho na panela.

4. elif v < 0.10:

Volatilidade abaixo de 10% é baixa vol.

Momento tranquilo, quase tedioso.

5. else: → “neutro”

Quando nada encaixa em nenhuma regra.

In [3]:
import numpy as np
def calcular_retornos(precos):
# pct_change()  calcula a variação percentual entre cada linha e a linha anterior exemplo:Se ontem valia 100 e hoje vale 105, pct_change() devolve (105 - 100) / 100 = 0.05, ou seja, 5%.
# fillna(0)  substitui valores ausentes por algo. Nesse caso, por 0
    retornos = precos.pct_change().fillna(0)
    return retornos

ret = calcular_retornos(precos)
# print(ret.head())
# print(ret.tail())
#  Volatilidade anualizada baseada em janela de retornos diários
def calcular_volatilidade(retornos, janela=30):
# basicamente ele pega uma janela de 30 dia  • No dia 30, ela olha os retornos dos dias 1 a 30. No dia 31, ela olha dos dias 2 a 31.No dia 32, ela olha dos dias 3 a 32.E a cada janela ele faz  vezes a raiz de 252
    return retornos.rolling(janela, min_periods=1).std() * np.sqrt(252)
vol = calcular_volatilidade(ret)
# print(vol.tail())
#  Calcula o drawdown de uma série de preços acumulados (portfólio ou índice).
def calcular_drawdown(series):
    """
Calcula o drawdown usando retornos diários.

    Explicação detalhada:
    - (1 + series).cumprod(): transforma retornos em valor acumulado.
    - cummax(): descobre o topo histórico a cada dia.
    - drawdown = diferença percentual entre valor atual e topo histórico.

    """
    cumulative = (1 + series).cumprod()
    max_cum = cumulative.cummax()
    dd = (cumulative - max_cum) / max_cum
    return dd
#  pega cada linha (cada dia) calcula a média dos retornos entre todos os ativos
indice_global = ret.mean(axis=1)  
# calculando o drawdown da sua carteira média
drawdown = calcular_drawdown(indice_global)

def classificar_regime(retornos, volatilidade, drawdown):
    """
    Classifica o regime de mercado com regras simples.
    bull  mercado otimista preços subindo sentimento positiv riscos menores (na média) a tradução literal seria touro
    bear = urso  • mercado pessimista preços caindo quedas fortes clima de medo
    """
    regimes = []

    for i in range(len(retornos)):
        r = retornos[i]
        v = volatilidade[i]
        d = drawdown[i]

        if d < -0.20:
            regimes.append("bear")
        elif r > 0.003 and d > -0.05:
            regimes.append("bull")
        elif v > 0.25:
            regimes.append("alta_vol")
        elif v < 0.10:
            regimes.append("baixa_vol")
        else:
            regimes.append("neutro")

    return regimes

r_media = ret.mean(axis=1)        # retorno médio diário do mercado
v_media = vol.mean(axis=1)        # volatilidade média
dd = drawdown                     # já calculado

regimes = classificar_regime(r_media.values, v_media.values, dd.values)





C:\Users\olazu\AppData\Local\Temp\ipykernel_15992\2860437914.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = precos.pct_change().fillna(0)


In [4]:
import torch
import numpy as np

# BR ETFs / Ações → cluster 0
# ETFs EUA → cluster 1
# Criptomoedas → cluster 2
# Ouro (GLD) → cluster 3
# Renda Fixa (TLT) → cluster 4

CLUSTERS = {
    "BOVA11.SA": 0,
    "SMAL11.SA": 0,
    "IVVB11.SA": 0,
    "PETR4.SA": 0,
    "VALE3.SA": 0,
    "ITUB4.SA": 0,

    "SPY": 1,
    "QQQ": 1,
    "IWM": 1,
    "EEM": 1,

    "GLD": 3,
    "TLT": 4,

    "BTC-USD": 2,
    "ETH-USD": 2,
    "BNB-USD": 2,
    "SOL-USD": 2,
    "ADA-USD": 2
}
# t seria cada item de clusters porque se eu coloca clusters for in tickers ele vai percorre a chave e nao valor 
cluster_ids = [CLUSTERS[t] for t in TICKERS]

regime_map = {
    "bull": 0,
    "bear": 1,
    "alta_vol": 2,
    "baixa_vol": 3,
    "neutro": 4
}

regime_ids = [regime_map[r] for r in regimes]


# --- SPLIT TREINO / TESTE ---

N = len(ret)
split_idx = int(N * 0.8)

# treino (80%)
ret_train = ret.iloc[:split_idx]
r_media_train = r_media.iloc[:split_idx]
v_media_train = v_media.iloc[:split_idx]
dd_train = dd.iloc[:split_idx]
regime_ids_train = regime_ids[:split_idx]

# teste (20%)
ret_test = ret.iloc[split_idx:]
r_media_test = r_media.iloc[split_idx:]
v_media_test = v_media.iloc[split_idx:]
dd_test = dd.iloc[split_idx:]
regime_ids_test = regime_ids[split_idx:]

def discretizar_estado_financeiro(pesos, regime_id, cluster_ids, r_media, v_media, dd_dia):
    """
    Monta o vetor de estado para o RL no mercado financeiro.

    - pesos: vetor com alocações atuais
    - regime_id: estado do mercado (0 a 4)
    - cluster_ids: vetor com os clusters fixos dos ativos
    - r_media: retorno médio do mercado no dia
    - v_media: volatilidade média do mercado no dia

    Retorna:
        Tensor shape [1, NUM_ATIVOS + 1 + 1 + NUM_ATIVOS]
        (pesos + r_media + v_media + cluster_ids)
    """

    features = np.concatenate([
    # round arredondada pesos para 4 casa decimal 
        np.round(pesos, 4),
        [r_media],
        [v_media],
        [dd_dia],
        np.array(cluster_ids, dtype=float)
    ])
    # SEGURANÇA: Garante que não há NaNs ou Infinitos que quebrariam a Rede Neural
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
# tensor = array so que do pytorch então a gente transforma em um tensor do pytoch as features e  unsqueeze(0) → cria uma dimensão de batc ou seja [1,N] N seria total de features
    return torch.FloatTensor(features).unsqueeze(0)


Significado:

Drawdown é geralmente definido como (preço atual / topo anterior) - 1

Logo, drawdown é zero ou negativo

Quanto mais negativo, pior

O modelo está punindo valores de drawdown mais profundos.
E sim: normalmente esse drawdown se refere à carteira, não à ação individual.

O topo histórico

np.abs: módulo disso (compra e venda contam igual)

np.sum: soma todas as mudanças

Exemplo:

Pesos antigos: [0.2, 0.3, 0.5]

Pesos novos: [0.1, 0.5, 0.4]

Diferenças: [-0.1, 0.2, -0.1]
Módulo: [0.1, 0.2, 0.1]
Soma: 0.4 → esse é o turnover.
Depois:

custo_transacao = lambda_tc * turnover


Se o turnover é 0.4 e lambda_tc = 0.001, o custo é:
0.001 * 0.4 = 0.0004

Simplificando: quanto mais você gira carteira, maior o custo.

In [5]:
import numpy as np

def calcular_recompensa_portfolio(
    pesos_antigos,
    pesos_novos,
    retornos_dia,
    drawdown_dia,
    lambda_dd=0.2,
    lambda_tc=0.001
):
    """
    Recompensa do agente no contexto de portfólio.

    - pesos_antigos: vetor de alocações antes da ação
    - pesos_novos: vetor de alocações depois da ação
    - retornos_dia: vetor de retornos dos ativos no dia (ex: ret.iloc[t].values)
    - drawdown_dia: drawdown do 'índice global' no dia (escalar)
    - lambda_dd: peso da penalização de drawdown
    - lambda_tc: peso do custo de transação (turnover)

    Retorna:
        escalar (recompensa)
    """

    # 1) Retorno do portfólio no dia
    # np.dot faz vira um unico número  pesos_novos é um vetor, tipo [0.3, 0.5, 0.2] retornos_dia é outro vetor, tipo [0.01, -0.02, 0.005] vira 0.3*0.01 + 0.5*(-0.02) + 0.2*(0.005)
    r_p = float(np.dot(pesos_novos, retornos_dia))

    # Proteção para não dar log de número <= 0
    if 1 + r_p <= 0:
        log_ret = -10.0    # punição pesada (quase "quebrou a carteira")
    else:
        log_ret = np.log(1 + r_p)

    # 2) Penalidade de drawdown (só quando negativo)
    dd = float(drawdown_dia)
    if dd < 0:
        penal_dd = lambda_dd * (dd ** 2)   # dd é negativo, dd**2 é positivo
    else:
        penal_dd = 0.0

    # 3) Custo de transação (turnover L1)
    # pesos_novos - pesos_antigos: quanto o agente mudou cada alocação
    turnover = float(np.sum(np.abs(pesos_novos - pesos_antigos)))
    custo_transacao = lambda_tc * turnover
# Retorno logarítmico Menos punição por drawdown Menos custo por mexer nos pesos
    recompensa = log_ret - penal_dd - custo_transacao
    return recompensa


t = 200  # ou ep % len(ret), dependendo do mapeamento

pesos = np.ones(NUM_ATIVOS) / NUM_ATIVOS
novos_pesos = pesos.copy()
novos_pesos[0] += 0.05
novos_pesos[1] -= 0.05

r_t = ret.iloc[t].values
dd_t = dd.iloc[t]


retornos_dia = ret.iloc[t].values          # vetor de retornos dos 17 ativos
drawdown_dia = dd.iloc[t]                  # drawdown do índice global
# lambda_dd (penalidade de drawdown) Serve para dizer o quanto você quer punir o agente por deixar a carteira cair longe do topo histórico.
# lambda_tc (custo de transação) Regula quanto você quer desencorajar o agente a ficar trocando pesos toda hora.
recompensa = calcular_recompensa_portfolio(
    pesos_antigos=pesos,
    pesos_novos=novos_pesos,
    retornos_dia=retornos_dia,
    drawdown_dia=drawdown_dia,
    lambda_dd=0.2,
    lambda_tc=0.001
)


recompensa_teste = calcular_recompensa_portfolio(pesos, novos_pesos, r_t, dd_t)
print(recompensa_teste)




-0.008030805295683797


Próximo passo:

Implementar o novo aplicar_acao_portfolio()

Escolha A ou B:

A) Apenas pesos positivos (carteira normal, 0% a 100%)

mais simples

mais realista para investidores normais

mais fácil do RL aprender

menor risco de explosão

B) Permitir pesos negativos (short selling)

mais realista para fundos multimercado

muito mais difícil para o RL

pode gerar comportamento agressivo

drawdown pode explodir se não calibrar bem 
O que o "Peso" significa para o Robô?
O peso é simplesmente "quanto da minha carteira eu coloco nisso?".

Cenário A: Pesos Positivos (Apenas Long)
Peso 0.5 (50%): "Vou colocar metade do dinheiro aqui porque acho que vai subir."

Peso 0.0 (0%): "Não quero ter isso na mão. Acho que vai cair ou ficar ruim." 

Cenário B: Pesos Negativos (Short)
Peso -0.5 (-50%): "Eu não tenho isso, mas vou vender emprestado porque tenho certeza que vai despencar. Quero ganhar dinheiro com a destruição desse preço."
A Venda Imediata
Você sai da loja e, imediatamente, vende esse iPhone para alguém na rua pelo preço de hoje: R$ 5.000.

Seu bolso: Agora você tem R$ 5.000 em dinheiro.

Sua dívida: Você ainda deve um iPhone novo para a loja.

Passo 3: A Espera (A Aposta)
Você espera uma semana com o dinheiro na mão.

Passo 4: O Fechamento (Dois cenários)
Cenário Bom (Você acertou): O preço do iPhone realmente caiu para R$ 3.000.

Você pega seus R$ 5.000.

Compra um iPhone novo por R$ 3.000.

Devolve o iPhone para a loja.

Lucro: R$ 5.000 (venda) - R$ 3.000 (recompra) - R$ 50 (aluguel) = R$ 1.950 no seu bolso. Isso é o Peso Negativo: Você ganhou dinheiro porque o preço caiu.

Cenário Ruim (O Risco Infinito): O dólar disparou e o iPhone subiu para R$ 8.000.

A loja exige o iPhone de volta.

Você tem que comprar um por R$ 8.000.

Você só tinha R$ 5.000 da venda original.

Prejuízo: Você tem que tirar R$ 3.000 do seu próprio bolso para cobrir a diferença.

In [6]:
#Ação 0: aumentar o peso do ativo Ação 1: diminuir o peso do ativo Isso quer dizer que, para cada ativo, existem duas ações possíveis:
AÇOES_POR_ATIVO = 2     
DELTA_PESO = 0.02          # passo pequeno de ajuste (2%)

def aplicar_acao_portfolio(pesos, action, passo=0.02, max_weight=0.2):
    """
    Ajusta os pesos dado um índice de ação discreta.

    - pesos: vetor de alocações atuais
    - action: inteiro [0, NUM_ATIVOS * 2)
        se action < NUM_ATIVOS  -> aumenta peso do ativo
        se action >= NUM_ATIVOS -> diminui peso do ativo
    - passo: tamanho do ajuste em cada ação
    - max_weight: limite máximo por ativo (ex: 0.2 = 20%)

    Retorna:
        novos_pesos: vetor normalizado, long-only, com cap de peso.
    """
    pesos = pesos.copy()
    num_assets = len(pesos)

    idx = action % num_assets
    direcao = action // num_assets  # 0 = aumenta, 1 = diminui

    if direcao == 0:
        pesos[idx] += passo
    else:
        pesos[idx] -= passo

    # impede pesos negativos
    pesos = np.clip(pesos, 0.0, None)

    soma = pesos.sum()
    if soma == 0:
        # fallback: volta pro equal-weight
        pesos = np.ones(num_assets) / num_assets
    else:
        pesos = pesos / soma

    # aplica limite máximo por ativo
    pesos = np.minimum(pesos, max_weight)

    # renormaliza depois do cap
    soma2 = pesos.sum()
    if soma2 == 0:
        pesos = np.ones(num_assets) / num_assets
    else:
        pesos = pesos / soma2

    return pesos


pesos = np.ones(NUM_ATIVOS) / NUM_ATIVOS
acao_teste = 0  # mexe no ativo 0 aumentando

novos = aplicar_acao_portfolio(pesos, acao_teste)

print("Soma:", novos.sum())
print("Min:", novos.min())
print("Max:", novos.max())
print(novos[:5])


Soma: 1.0000000000000002
Min: 0.05767012687427913
Max: 0.07727797001153404
[0.07727797 0.05767013 0.05767013 0.05767013 0.05767013]


In [7]:
pesos = np.ones(NUM_ATIVOS) / NUM_ATIVOS

for a in range(10):
    pesos = aplicar_acao_portfolio(pesos, action=a)
    print(f"Ação {a} | soma={pesos.sum():.6f} | min={pesos.min():.4f} | max={pesos.max():.4f}")


Ação 0 | soma=1.000000 | min=0.0577 | max=0.0773
Ação 1 | soma=1.000000 | min=0.0565 | max=0.0761
Ação 2 | soma=1.000000 | min=0.0554 | max=0.0750
Ação 3 | soma=1.000000 | min=0.0543 | max=0.0740
Ação 4 | soma=1.000000 | min=0.0533 | max=0.0729
Ação 5 | soma=1.000000 | min=0.0522 | max=0.0718
Ação 6 | soma=1.000000 | min=0.0512 | max=0.0708
Ação 7 | soma=1.000000 | min=0.0502 | max=0.0698
Ação 8 | soma=1.000000 | min=0.0492 | max=0.0688
Ação 9 | soma=1.000000 | min=0.0483 | max=0.0679


In [8]:
__all__ = [
    "TICKERS", "NUM_ATIVOS",
    "ret", "r_media", "v_media", "dd",
    "regimes", "regime_ids", "cluster_ids",
    "ret_train", "r_media_train", "v_media_train", "dd_train", "regime_ids_train",
    "ret_test", "r_media_test", "v_media_test", "dd_test", "regime_ids_test",
    "discretizar_estado_financeiro",
    "calcular_recompensa_portfolio",
    "aplicar_acao_portfolio"
]
